In [1]:
import pandas as pd 
import numpy as np
import japanize_matplotlib
from sklearn import linear_model
%matplotlib inline
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import svm,metrics
from sklearn.metrics import confusion_matrix # 混合行列
from sklearn.decomposition import PCA #主成分分析
from sklearn.linear_model import LogisticRegression # ロジスティック回帰
from sklearn.neighbors import KNeighborsClassifier # K近傍法
from sklearn.svm import SVC # サポートベクターマシン
from sklearn.tree import DecisionTreeClassifier # 決定木
from sklearn.ensemble import RandomForestClassifier # ランダムフォレスト
from sklearn.ensemble import AdaBoostClassifier # AdaBoost
from sklearn.naive_bayes import GaussianNB # ナイーブ・ベイズ
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
# from sklearn.lda import LDA # 線形判別分析
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA
from sklearn.model_selection import GridSearchCV
from tqdm import tqdm
from sklearn.metrics import f1_score, r2_score, classification_report
# from sklearn.qda import QDA # 二次判別分析
from seglearn.pipe import Pype
from seglearn.split import temporal_split
from seglearn.transform import FeatureRep, SegmentX
from seglearn.transform import SegmentXY, last, middle, mean
from seglearn.feature_functions import all_features, minimum, maximum,  mean,  std, kurt, mean_diff, median, willison_amplitude, variation
import math
from matplotlib.font_manager import FontProperties
import random
import pickle
import seaborn as sns

In [2]:
# 瞬きの修正
def blinkFix(data):
    print("start blinkFix--------------")
    # 瞬き認識
    blink = 0.7
    blinkArray = []
    for index, row in data.loc[data['eyeBlink'] > blink].iterrows():
            blinkArray.append(index)
   # 瞬きの始まりと終わり
    blinkStartIndexs = []
    blinkEndIndexs = []
    for i in range(len(blinkArray)):
        blinkStart = True
        blinkEnd = True
        if i - 1 >= 0:
            if blinkArray[i] - 1 == blinkArray[i-1]:
                blinkStart = False
        if i + 1 < len(blinkArray):
            if blinkArray[i] + 1 == blinkArray[i+1]:
                blinkEnd = False       
        if blinkStart:
            blinkStartIndexs.append(i)
        if blinkEnd:
            blinkEndIndexs.append(i) 
    blink_data = data.copy()
    startOffset = -5
    endOffset = 15
    for i in range(len(blinkStartIndexs)):
        blinkStartIndex = blinkArray[blinkStartIndexs[i]]
        blinkEndIndex = blinkArray[blinkEndIndexs[i]]
        if blinkStartIndex+startOffset < 0:
            blinkstart = 0
        else:
            blinkstart = blinkStartIndex+startOffset
        
        if blinkEndIndex+endOffset >= len(blink_data) :
            blinkend = len(blink_data)-1
        else:
            blinkend = blinkEndIndex+endOffset
        
        for j in range(blinkstart, blinkend):
            if j  >= 0 and  j < len(blink_data):
                d = blinkstart-blinkend
                ax = blink_data.iloc[blinkstart,:]['centerEyeLookAtPoint-x']-blink_data.iloc[blinkend,:]['centerEyeLookAtPoint-x']
                bx = blinkstart*blink_data.iloc[blinkend,:]['centerEyeLookAtPoint-x'] - blinkend*blink_data.iloc[blinkstart,:]['centerEyeLookAtPoint-x']
                ay = blink_data.iloc[blinkstart,:]['centerEyeLookAtPoint-y']-blink_data.iloc[blinkend,:]['centerEyeLookAtPoint-y']
                by = blinkstart*blink_data.iloc[blinkend,:]['centerEyeLookAtPoint-y'] - blinkend*blink_data.iloc[blinkstart,:]['centerEyeLookAtPoint-y']
                x = (ax*j + bx)/d
                y = (ay*j + by )/d
                blink_data.loc[blink_data.index == j, 'centerEyeLookAtPoint-x'] = x
                blink_data.loc[blink_data.index == j, 'centerEyeLookAtPoint-y'] = y
#      確認グラフ
#     beforFix = data.iloc[blinkArray[blinkStartIndexs[5]]-15:blinkArray[blinkEndIndexs[5]]+25,:].copy()
#     afterFix = blink_data.iloc[blinkArray[blinkStartIndexs[5]]-15:blinkArray[blinkEndIndexs[5]]+25,:].copy()
#     from matplotlib import pyplot as plt
#     plt1 = plt
#     plt1.plot(beforFix.index, beforFix['centerEyeLookAtPoint-x'], label="beforFix")
#     plt1.plot(afterFix.index, afterFix['centerEyeLookAtPoint-x'], label="afterFix")
#     plt1.ylabel('centerEyeLookAtPoint-x')
#     h1, l1 = plt1.axes().get_legend_handles_labels()
#     plt2 = plt1.twinx()
#     plt2.plot(afterFix.index, afterFix['eyeBlink'], c="r", label="eyeBlink", linestyle="dashed")
#     plt2.set_ylabel('eyeBlink')
#     h2, l2 = plt2.get_legend_handles_labels()
#     plt.legend(h1+h2, l1+l2, loc='upper left')
#     print(type(plt2))
#     plt1.show()
    
#     from matplotlib import pyplot as plt
#     plt1 = plt
#     plt1.plot(beforFix.index, beforFix['centerEyeLookAtPoint-y'], label="beforFix")
#     plt1.plot(afterFix.index, afterFix['centerEyeLookAtPoint-y'], label="afterFix")
#     plt1.ylabel('centerEyeLookAtPoint-y')
#     h1, l1 = plt1.axes().get_legend_handles_labels()
#     plt2 = plt1.twinx()
#     plt2.plot(afterFix.index, afterFix['eyeBlink'], c="r", label="eyeBlink", linestyle="dashed")
#     plt2.set_ylabel('eyeBlink')
#     h2, l2 = plt2.get_legend_handles_labels()
#     plt.legend(h1+h2, l1+l2, loc='upper left')
#     print(type(plt2))
#     plt1.show()
    
    print("end blinkFix--------------")
    return blink_data

In [3]:
def dataInit(data):
    data['eyeBlink'] = (data['rightEyeBlink'] + data['leftEyeBlink'] )/2
    data = data[['timestamp', 'centerEyeLookAtPoint-x', 'centerEyeLookAtPoint-y',  'eyeBlink', 'operationType', 'operationPoint-x', 'operationPoint-y']].copy()
    print("start dataInit--------------")
    blink_data = blinkFix(data)
    #キャリブレーション 
    target_x = blink_data.iloc[0,:]['operationPoint-x']
    target_y = blink_data.iloc[0,:]['operationPoint-y']
    for index, row in blink_data.iterrows():
        if target_x != row['operationPoint-x'] or target_y != row['operationPoint-y']:
            center_end = index-1
            break
    if center_end-150 < 0:
        print("center_end-150<0")
    else:
        center_start = center_end-150
    eyePointMean_x= blink_data.iloc[center_start:center_end,:]['centerEyeLookAtPoint-x'].mean()
    eyePointMean_y = blink_data.iloc[center_start: center_end,:]['centerEyeLookAtPoint-y'].mean()
    cal_x = target_x - eyePointMean_x
    cal_y = target_y - eyePointMean_y
    blink_data['centerEyeLookAtPoint-x'] = blink_data['centerEyeLookAtPoint-x'] + cal_x 
    blink_data['centerEyeLookAtPoint-y'] = blink_data['centerEyeLookAtPoint-y'] + cal_y 
#     centerキャリブレーションの確かめグラフ
#     cal_data = blink_data.iloc[center_start:center_end, :].copy()
#     data_ = data.iloc[center_start:center_end,:].copy()
#     plt.scatter(cal_data.index, cal_data['centerEyeLookAtPoint-y'], s=1, label="targetPoint-y(afterFix)")
#     plt.scatter(cal_data.index, cal_data['operationPoint-y'], s=1, label="operationPoint-x")
#     plt.scatter(data_.index, data_['centerEyeLookAtPoint-y'], s=1, label="targetPoint-y(beforFix)")
#     plt.xticks(rotation=40)
#     plt.legend(loc="upper left", fontsize=14) 
#     plt.show()
    #分類に使用するデータ     
    print("end dataInit--------------")
    return blink_data.iloc[center_end+1:, :]

In [4]:
# 画面をsplitx×splityに分割してカラム名columnにラベル付（0は例外）
# data:ラベル付をするdataframe
# column:ラベル付するカラム名
# splitx,splity:画面を分割する数
# shift:nextColumnをどれだけずらすか（プラス）
def c_labeling(data, splitx, splity, shift):
    print("start c_labeling--------------")
    column = 'area'
    nextColumn = 'next_' + column
    intention = 'switch'
    width = 414
    height = 896
    x_d = width/splitx
    y_d = height/splity
    data[column] = 0
    labelWidth = 5#ラベル付の幅
    
#     columnにラベル付
    for index, row in data.iterrows():
        if(row['operationPoint-x'] != 0 and row['operationPoint-y'] != 0):
            label =  splitx * (math.floor(row['operationPoint-y']/y_d)) + math.floor(row['operationPoint-x']/x_d)+1
            data.loc[data.index == index, column] = label
    print("finish 1/3")
    
#     nextColumnとintentionにラベル付
    data[nextColumn] = 0
    data[intention] = 0
    for index, row in data.iterrows():
        if index < len(data)-1:
            if data.at[index, column] != data.at[index+1,column] and data.at[index+1, column] != 0:
                if index-(shift-1) >= 0:
                    data.at[index-(shift-1), nextColumn] = data.at[index+1, column]
                for i in range(index-(shift-1)-(labelWidth-1), index+1-(shift-1)):
                    if i >= 0:
                        data.at[i, intention] = 1
    print("finish 2/3")

    print("finish 3/3")
    print("end c_labeling--------------")

In [5]:
# セグメンテーション，操作前を一定フレームごとに分割
# width:何フレームでセグメントするか
# segmentIx = []，segmentIy = []（操作直前か，そうでないかの分類用）（直前の場合）
# segmentx = []，segmenty = []（操作直前の操作座標の全てのラベルの分類用）
# intention:操作直前のラベル付がされたカラム名
# nextColumn:操作座標のラベル付がされたカラム名
# labelNum:操作座標が何分割されているか
def segmentation(data, width, segmentIx, segmentIy, segmentx, segmenty):
    print("start segmentation--------------")
    column = 'area'
    nextColumn = 'next_' + column
    intention = 'switch'
    # segmentのwidth
    shift = 5
    for index, row in data.iterrows():
        if index >= width-1and index < len(data)-1:
            data_ = data.iloc[index-(width-1):index+1,:].copy()
            segmentIx.append(data_[['centerEyeLookAtPoint-x', 'centerEyeLookAtPoint-y']].values)
            segmentIy.append(row[intention])
            if row[nextColumn] != 0:
                for i in range(index-shift-1, index+1):
                    data_ = data.iloc[i-(width-1): i+1,:].copy()
                    segmentx.append(data_[['centerEyeLookAtPoint-x', 'centerEyeLookAtPoint-y']].values)
                    segmenty.append(row[nextColumn])
    print(len(segmentx))     
    print("end segmentation--------------")

In [6]:
# セグメントのラベル数を揃える
def adjust_cLabel(segmentIx, segmentIy, segmentx, segmenty):
    print("start adjust_segment--------------")
    onelist = [i for i, x in enumerate(segmentIy) if x == 1]
    zerolist = [j for j, y in enumerate(segmentIy) if y == 0]
    random.shuffle(zerolist)
    dI = len(zerolist)-len(onelist)
    print("onelist: {0}, zerolist: {1}, dI: {2}".format(len(onelist), len(zerolist), dI))
    zerolist_ = zerolist[:dI]
    print("zerolist_:{0}".format(len(zerolist_)))
    for k in sorted(zerolist_, reverse=True):
        segmentIy.pop(k)
        segmentIx.pop(k)
    print("finish 1/2")           
    print("min:{0}, max:{1}".format(min(segmenty), max(segmenty)))
    countlist = {}
    for i in range(len(segmenty)):
        if segmenty[i] in countlist:
            countlist[segmenty[i]] += 1
        else:
            countlist[segmenty[i]] = 1
        
    countmin = countlist[min(countlist)]
    print(countmin) 
    for key, val in countlist.items():
        if val > countmin:
            print("key:{0}, val:{1}".format(key, val))
            indexlist = [m for m, z in enumerate(segmenty) if z == key]
            random.shuffle(indexlist)
            d = countlist[key] - countmin
            indexlist_ = indexlist[:d]
            for n in sorted(indexlist_, reverse=True):
                segmenty.pop(n)
                segmentx.pop(n)
    print("finish 2/2")
    print("end adjust_segment--------------")

In [7]:
def heatmap(clf, segmentx, segmenty):
    y, y_p = clf.transform_predict(segmentx, segmenty)
    labels = sorted(list(set(y)))
    cm_ = confusion_matrix(y,  y_p, labels=labels)
    cm = cm_.astype(np.float32)
    for i in range(len(cm)):
        sum = 0
        for j in range(len(cm[0])):
            sum += cm[i][j]
        cm[i] = (cm[i]*100)/sum
#     cm = pd.DataFrame(cm, index=labels, columns=labels)
    sns.heatmap(cm, cmap='Blues', annot=True, fmt='.1f', vmin=0, vmax=100)
    plt.xlabel('予測ラベル')
    plt.ylabel('正解ラベル')
    plt.tight_layout()
    plt.rcParams["font.size"] = 10
    plt.rcParams['figure.figsize'] = (7.0, 6.0)
    plt.show()

In [8]:
# モデルの学習
# fts：特徴量（配列）
# classifer：モデル
# labelCount：1つのラベルの数
def classification(fts, classifier, segmentIx, segmentIy):
    clf = Pype([
                    ('features', FeatureRep(features=fts)),  # extracts features
                    ('rf', classifier)  #  ML algorithm of sklearn
                ])

    print(clf)
    segmentIx = np.array(segmentIx)
    X = segmentIx
    Y = segmentIy
    X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=0)
    print("count of data:", len(segmentIx))

    # # fit and score
    clf.fit(X_train, y_train)
    score = clf.score(X_test, y_test)
    print("score of test data：", score)
    score = clf.score(X_train, y_train)
    print("score of train data：", score)
    score = clf.score(X, Y)
    print("score of all data：", score)
    
    heatmap(clf, X_test, y_test)
    
    return clf

In [9]:
def classification_(clf, segmentx, segmenty):
    score= clf.score(segmentx, segmenty)
    print("count of data：{0}".format(len(segmentx)))
    print("clf_score: {0}".format(score))
    heatmap(clf, segmentx, segmenty)

In [10]:
def ftsScore(fts_name, fts_name_, fts_, classifier, segmentIx, segmentIy):
    segmentIx = np.array(segmentIx)
    X = segmentIx
    Y = segmentIy
    X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=0)
    result = []
    for i in range(len(fts_name)):
        fts = {fts_name[i]: fts_[i]}
        clf = Pype([
                        ('features', FeatureRep(features=fts)),  # extracts features
                        ('rf', RandomForestClassifier(n_estimators=20))  #  ML algorithm of sklearn
                    ])

        clf.fit(X_train, y_train)
        score1 = clf.score(X_train, y_train)
        score2 = clf.score(X_test, y_test)
        result.append([score1, score2])
    df_result = pd.DataFrame(result, columns=['train', 'test'], index=fts_name_).sort_values('test', ascending=False)
    plt.rcParams["font.size"] = 18
    df_result.plot(kind='bar', alpha=0.5, grid=True)
    return clf

In [11]:
def distance(x1, y1, x2, y2):
    disx = x1-x2
    disy = y1-y2
    dis = np.sqrt(disx**2 + disy**2)
    return dis
# 視線と操作座標の距離の確認用
def distSeg(data, width, segment, column):
    # segmentのwidth
    shift = width
    opeDist = 0
    for index, row in data.iterrows():
        if index < len(data)-1 and index > shift:
            if row[column] != data.iloc[index+1,:][column] and data.iloc[index+1,:][column] != 0:
                #next columnを追加
                data_ = data.loc[index-(shift-2):index+1,:].copy()
                dist = distance(data_['centerEyeLookAtPoint-x'], data.iloc[index+1,:]['operationPoint-x'], data_['centerEyeLookAtPoint-y'], data.iloc[index+1,:]['operationPoint-y'])  
                data_['distance'] = dist
                data_['opeDistance'] = 0
                for index_, row_ in data_.iterrows():
                    if not(row_['operationPoint-x'] == 0 and row_['operationPoint-y'] == 0):
                        opeDist = distance(data_.at[index_, 'operationPoint-x'], data.iloc[index+1,:]['operationPoint-x'], data_.at[index_, 'operationPoint-y'], data.iloc[index+1,:]['operationPoint-y'])             
                    data_.at[index_, 'opeDistance'] = opeDist
                data_['nextOpePoint-x'] = data.iloc[index+1,:]['operationPoint-x']
                data_['nextOpePoint-y'] = data.iloc[index+1,:]['operationPoint-y']
                segment.append(data_[['centerEyeLookAtPoint-x', 'centerEyeLookAtPoint-y', 'operationPoint-x', 'operationPoint-y', 'distance', 'opeDistance', 
                                      'nextOpePoint-x', 'nextOpePoint-y']])

In [12]:
# データ平滑化
def dataRolling(data, window):
    data['centerEyeLookAtPoint-y'] = data['centerEyeLookAtPoint-y'].rolling(window=window, min_periods=1).mean()
    data['centerEyeLookAtPoint-x'] = data['centerEyeLookAtPoint-x'].rolling(window=window, min_periods=1).mean()

In [13]:
def direction(data):
    # segmentのwidth
    shift = width
    sx = data.iloc[0,:]['operationPoint-x']
    sy = data.iloc[0,:]['operationPoint-y']
    data['direction_ope'] = 0
    data['direction_eye'] = 0
    for index, row in data.iterrows():
        if index < len(data)-2:
            dire_ope = math.atan2(sy-data.iloc[index+1,:]['operationPoint-y'], sx-data.iloc[index+1,:]['operationPoint-x'])
            dire_eye = math.atan2(row['centerEyeLookAtPoint-y']-data.iloc[index+1,:]['centerEyeLookAtPoint-y'], row['centerEyeLookAtPoint-x']-data.iloc[index+1,:]['centerEyeLookAtPoint-x'])
            data.at[index, 'direction_ope'] = math.degrees(dire_ope)
            data.at[index, 'direction_eye'] = math.degrees(dire_eye)
            sx = row['centerEyeLookAtPoint-x']
            sy = row['centerEyeLookAtPoint-y']

In [14]:
# データの事前処理
# mode：セグメントモード(0：学習時，1：予測時)
# csv_data_s：csvデータ(dataframe)の配列
# column：操作座標ラベルのカラム名
# splitx，splity：画面分割数
# shift：ラベルをずらす数
# width：セグメントの幅
def pre_processing(mode, csv_datas, segmentIx, segmentIy, segmentx, segmenty, splitx, splity, shift, width):
    data = pd.DataFrame(index=[])
    for csv_data in csv_datas:
        data_ = dataInit(csv_data)
        dataRolling(data_, 30)
        c_labeling(data_, splitx, splity, shift)
        segmentation(data_, width, segmentIx, segmentIy, segmentx, segmenty)
        data = pd.concat([data, data_])
    data = data.reset_index(drop=True)
    if mode == 0:
        adjust_cLabel(segmentIx, segmentIy, segmentx, segmenty)
    return data

In [15]:
# データ処理
csv_data1_ = pd.read_csv('./csvFile/saki/saki_2020121207440260200_home.csv')
csv_data2_ = pd.read_csv('./csvFile/saki/saki_2020121208154033100_home.csv')
csv_data3_ = pd.read_csv('./csvFile/saki/saki_2020121213360798900_home.csv')
csv_data4_ = pd.read_csv('./csvFile/saki/saki_2020121807340371200_home.csv')
csv_data5_ = pd.read_csv('./csvFile/saki/saki_2020121816101633100_home.csv')
csv_data6_ = pd.read_csv('./csvFile/saki/saki_2020122007312691200_home.csv')
# 学習用データ
# csv_datas = [csv_data1_, csv_data2_, csv_data3_, csv_data4_, csv_data5_]
# segIx4_0 = []
# segIy4_0 = []
# segx4_0 = []
# segy4_0 = []
# data4_0 = pre_processing(0, csv_datas, segIx4_0, segIy4_0, segx4_0, segy4_0, 2, 2, 0, 150)
# # 確かめデータ
# csv_datas_ = [csv_data6_]
# segIx4_0_ = []
# segIy4_0_ = []
# segx4_0_ = []
# segy4_0_ = []
# data4_0_ = pre_processing(1, csv_datas_, segIx4_0_, segIy4_0_, segx4_0_, segy4_0_, 2, 2, 0, 150)

In [16]:
csv_datas = [csv_data1_, csv_data2_, csv_data3_, csv_data4_, csv_data5_]
segIx4_60 = []
segIy4_60 = []
segx4_60 = []
segy4_60 = []
data4_60 = pre_processing(0, csv_datas, segIx4_60, segIy4_60, segx4_60, segy4_60, 2, 2, 60, 150)
# 確かめデータ
csv_datas_ = [csv_data6_]
segIx4_60_ = []
segIy4_60_ = []
segx4_60_ = []
segy4_60_ = []
data4_60_ = pre_processing(1, csv_datas_, segIx4_60_, segIy4_60_, segx4_60_, segy4_60_, 2, 2, 60, 150)

start dataInit--------------
start blinkFix--------------
end blinkFix--------------
end dataInit--------------
start c_labeling--------------
finish 1/3
finish 2/3
finish 3/3
end c_labeling--------------
start segmentation--------------
1442
end segmentation--------------
start dataInit--------------
start blinkFix--------------
end blinkFix--------------
end dataInit--------------
start c_labeling--------------
finish 1/3
finish 2/3
finish 3/3
end c_labeling--------------
start segmentation--------------
3045
end segmentation--------------
start dataInit--------------
start blinkFix--------------
end blinkFix--------------
end dataInit--------------
start c_labeling--------------
finish 1/3
finish 2/3
finish 3/3
end c_labeling--------------
start segmentation--------------
4515
end segmentation--------------
start dataInit--------------
start blinkFix--------------
end blinkFix--------------
end dataInit--------------
start c_labeling--------------
finish 1/3
finish 2/3
finish 3/3
en

In [17]:
# indexlist = [m for m, z in enumerate(segy4_60) if z == 'NaN']
# print(len(indexlist))
# print(segy4_60[indexlist[0]])
# data4_60[data4_60['next_area'].isnull()]
data4_60[data4_60['area'].isnull()]

,timestamp,centerEyeLookAtPoint-x,centerEyeLookAtPoint-y,eyeBlink,operationType,operationPoint-x,operationPoint-y,area,next_area,switch
41870,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,1.0
41871,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
41872,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
41873,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
41874,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [18]:
for index, row in data4_60.iterrows():
    if row['next_area'] != 0:
        if row['next_area'] != data4_60.iloc[index+60, :]['area']:
            print(index)

41870
41871
41872
41873
41874


In [19]:
data4_60.iloc[41869:41870,:]
data4_60.at[0]

TypeError: _get_value() missing 1 required positional argument: 'col'

In [ ]:
csv_datas = [csv_data1_, csv_data2_, csv_data3_, csv_data4_, csv_data5_]
segIx4_300 = []
segIy4_300 = []
segx4_300 = []
segy4_300 = []
data4_300 = pre_processing(0, csv_datas, segIx4_300, segIy4_300, segx4_300, segy4_300, 2, 2, 300, 150)
# 確かめデータ
csv_datas_ = [csv_data6_]
segIx4_300_ = []
segIy4_300_ = []
segx4_300_ = []
segy4_300_ = []
data4_300_ = pre_processing(1, csv_datas_, segIx4_300_, segIy4_300_, segx4_300_, segy4_300_, 2, 2, 300, 150)

In [ ]:
csv_datas = [csv_data1_, csv_data2_, csv_data3_, csv_data4_, csv_data5_]
segIx28_0 = []
segIy28_0 = []
segx28_0 = []
segy28_0 = []
data28_0 = pre_processing(0, csv_datas, segIx28_0, segIy28_0, segx28_0, segy28_0, 4, 7, 0, 150)
# 確かめデータ
csv_datas_ = [csv_data6_]
segIx28_0_ = []
segIy28_0_ = []
segx28_0_ = []
segy28_0_ = []
data28_0_ = pre_processing(1, csv_datas_, segIx28_0_, segIy28_0_, segx28_0_, segy28_0_, 4, 7, 0, 150)
csv_datas = [csv_data1_, csv_data2_, csv_data3_, csv_data4_, csv_data5_]
segIx28_60 = []
segIy28_60 = []
segx28_60 = []
segy28_60 = []
data28_60 = pre_processing(0, csv_datas, segIx28_60, segIy28_60, segx28_60, segy28_60, 4, 7, 60, 150)
# 確かめデータ
csv_datas_ = [csv_data6_]
segIx28_60_ = []
segIy28_60_ = []
segx28_60_ = []
segy28_60_ = []
data28_60_ = pre_processing(1, csv_datas_, segIx28_60_, segIy28_60_, segx28_60_, segy28_60_, 4, 7, 60, 150)
csv_datas = [csv_data1_, csv_data2_, csv_data3_, csv_data4_, csv_data5_]
segIx28_300 = []
segIy28_300 = []
segx28_300 = []
segy28_300 = []
data28_300 = pre_processing(0, csv_datas, segIx28_300, segIy28_300, segx28_300, segy28_300, 4, 7, 300, 150)
# 確かめデータ
csv_datas_ = [csv_data6_]
segIx28_300_ = []
segIy28_300_ = []
segx28_300_ = []
segy28_300_ = []
data28_300_ = pre_processing(1, csv_datas_, segIx28_300_, segIy28_300_, segx28_300_, segy28_300_, 4, 7, 300, 150)

In [ ]:
# 操作直前かそうでないかの分類
print("操作直前")
fts = {'var': variation, 'std': std, 'med': median}
classifier = RandomForestClassifier(n_estimators=20)
clfI4_0 = classification(fts, classifier, segIx4_0, segIy4_0)
# モデルを保存する
filename = './model/home_clfI4_0.sav'
pickle.dump(clfI4_0, open(filename, 'wb'))

# 操作座標の分類
print("操作領域")
fts = {'mean': mean, 'med': median, 'var': variation}
classifier = RandomForestClassifier(n_estimators=20)
clf4_0 = classification(fts, classifier, segx4_0, segy4_0)
# モデルを保存する
filename = './model/home_clf4_0.sav'
pickle.dump(clf4_0, open(filename, 'wb'))

In [ ]:
print("確認4_0")
classification_(clfI4_0, segIx4_0_, segIy4_0_)
classification_(clf4_0, segx4_0_, segy4_0_)

In [ ]:
# 操作直前かそうでないかの分類
print("操作直前")
fts = {'var': variation, 'std': std, 'med': median}
classifier = RandomForestClassifier(n_estimators=20)
clfI4_60 = classification(fts, classifier, segIx4_60, segIy4_60)
# モデルを保存する
filename = './model/home_clfI4_60.sav'
pickle.dump(clfI4_60, open(filename, 'wb'))

# 操作座標の分類
print("操作領域")
fts = {'mean': mean, 'med': median, 'var': variation}
classifier = RandomForestClassifier(n_estimators=20)
clf4_60 = classification(fts, classifier, segx4_60, segy4_60)
# モデルを保存する
filename = './model/home_clf4_60.sav'
pickle.dump(clf4_60, open(filename, 'wb'))

In [ ]:
print("確認4_60")
classification_(clfI4_60, segIx4_0_, segIy4_60_)
classification_(clf4_60, segx4_60_, segy4_60_)

In [ ]:
# 操作直前かそうでないかの分類
print("操作直前")
fts = {'var': variation, 'std': std, 'med': median}
classifier = RandomForestClassifier(n_estimators=20)
clfI4_300 = classification(fts, classifier, segIx4_300, segIy4_300)
# モデルを保存する
filename = './model/home_clfI4_300.sav'
pickle.dump(clfI4_300, open(filename, 'wb'))

# 操作座標の分類
print("操作領域")
fts = {'mean': mean, 'med': median, 'var': variation}
classifier = RandomForestClassifier(n_estimators=20)
clf4_300 = classification(fts, classifier, segx4_300, segy4_300)
# モデルを保存する
filename = './model/home_clf4_300.sav'
pickle.dump(clf4_300, open(filename, 'wb'))

In [ ]:
print("確認4_300")
classification_(clfI4_300, segIx4_300_, segIy4_300_)
classification_(clf4_300, segx4_300_, segy4_300_)

In [ ]:
# 操作直前かそうでないかの分類
print("操作直前")
fts = {'var': variation, 'std': std, 'med': median}
classifier = RandomForestClassifier(n_estimators=20)
clfI28_0 = classification(fts, classifier, segIx28_0, segIy28_0)
# モデルを保存する
filename = './model/home_clfI28_0.sav'
pickle.dump(clfI28_0, open(filename, 'wb'))

# 操作座標の分類
print("操作領域")
fts = {'mean': mean, 'med': median, 'var': variation}
classifier = RandomForestClassifier(n_estimators=20)
clf28_0 = classification(fts, classifier, segx28_0, segy28_0)
# モデルを保存する
filename = './model/home_clf28_0.sav'
pickle.dump(clf28_0, open(filename, 'wb'))

In [ ]:
print("確認28_0")
scoreI = clfI28_0.score(segIx28_0_, segIy28_0_)
print("count of data：{0}".format(len(segIx28_0_)))
print("clfI28_0score: {0}".format(scoreI))
heatmap(clfI28_0, segIx28_0_, segIy28_0_)
score = clfI28_0.score(segx28_0_, segy28_0_)
print("count of data：{0}".format(len(segx28_0_)))
print("clf28_0score: {0}".format(score))
heatmap(clf28_0, segx28_0_, segy28_0_)

In [ ]:
# 操作直前かそうでないかの分類
print("操作直前")
fts = {'var': variation, 'std': std, 'med': median}
classifier = RandomForestClassifier(n_estimators=20)
clfI28_60 = classification(fts, classifier, segIx28_60, segIy28_60)
# モデルを保存する
filename = './model/home_clfI28_60.sav'
pickle.dump(clfI28_60, open(filename, 'wb'))

# 操作座標の分類
print("操作領域")
fts = {'mean': mean, 'med': median, 'var': variation}
classifier = RandomForestClassifier(n_estimators=20)
clf28_60 = classification(fts, classifier, segx28_60, segy28_60)
# モデルを保存する
filename = './model/home_clf28_60.sav'
pickle.dump(clf28_60, open(filename, 'wb'))

In [ ]:
# 操作直前かそうでないかの分類
print("操作直前")
fts = {'var': variation, 'std': std, 'med': median}
classifier = RandomForestClassifier(n_estimators=20)
clfI28_300 = classification(fts, classifier, segIx28_300, segIy28_300)
# モデルを保存する
filename = './model/home_clfI28_300.sav'
pickle.dump(clfI28_300, open(filename, 'wb'))

# 操作座標の分類
print("操作領域")
fts = {'mean': mean, 'med': median, 'var': variation}
classifier = RandomForestClassifier(n_estimators=20)
clf28_300 = classification(fts, classifier, segx28_300, segy28_300)
# モデルを保存する
filename = './model/home_clf28_300.sav'
pickle.dump(clf28_300, open(filename, 'wb'))

In [ ]:
# cal_data = data.iloc[156:256, :].copy()
# plt.scatter(cal_data.index, cal_data['centerEyeLookAtPoint-y'], s=1, label="targetPoint-y")
# plt.scatter(cal_data.index, cal_data['operationPoint-y'], s=1, label="operationPoint-y")
# plt.xticks(rotation=40)
# # plt.legend(loc="upper left", fontsize=14) 
# plt.show()
# data1.iloc[204:250,:]
print(len([j for j, x in enumerate(segIy1) if x == 0]))
print(len([i for i, x in enumerate(segIy1) if x == 1]))
# print(len(segIx1))

In [ ]:
data1.iloc[195:240,:]

In [ ]:
# data.iloc[32500:32550,:]

In [ ]:
# data.iloc[430:440,:]
demoData = data1.copy()
dis_seg = []
width = 400
distSeg(demoData, width, dis_seg, 'area')
pltSeg = dis_seg[10]
from matplotlib import pyplot as plt
plt.plot(range(width), pltSeg['distance'],label="distance(nop-ce)")
plt.plot(range(width), pltSeg['opeDistance'], label="distance(nop-cop)")
plt.xticks(rotation=40)
plt.legend(loc="upper left", fontsize=14) 
plt.show()

In [ ]:
demoData = data1.copy()
dataRolling(demoData, 30)
dis_seg = []
width = 400
distSeg(demoData, width, dis_seg, 'area')
pltSeg = dis_seg[10]
from matplotlib import pyplot as plt
plt.plot(range(width), pltSeg['distance'],label="distance(nop-ce)")
plt.plot(range(width), pltSeg['opeDistance'], label="distance(nop-cop)")
plt.xticks(rotation=40)
plt.legend(loc="upper left", fontsize=14) 
plt.show()

# pltSeg = dis_seg[100]
# from matplotlib import pyplot as plt
# plt.scatter(range(width), pltSeg['distance'], s=1, label="distance")
# plt.scatter(range(width), pltSeg['opeDistance'], s=1, label="opeDistance")
# plt.xticks(rotation=40)
# plt.legend(loc="upper left", fontsize=14) 
# plt.show()

# pltSeg = dis_seg[700]
# from matplotlib import pyplot as plt
# plt.scatter(range(width), pltSeg['distance'], s=1, label="distance")
# plt.scatter(range(width), pltSeg['opeDistance'], s=1, label="opeDistance")
# plt.xticks(rotation=40)
# plt.legend(loc="upper left", fontsize=14) 
# plt.show()

In [ ]:
directionData = data1.copy()
direction(directionData)
s = 10
width = 300
pltSeg = directionData.iloc[s: s+width, :]
from matplotlib import pyplot as plt
plt.plot(pltSeg.index, pltSeg['direction_ope'], label="direction(op)")
plt.plot(pltSeg.index+1, pltSeg['direction_eye'],  label="direction(eye)")
plt.xticks(rotation=40)
plt.legend(loc="upper left", fontsize=14) 
plt.show()

In [ ]:
# 視線と操作位置の距離
# directionData = data1.copy()
# dataRolling(directionData, 20)
# direction(directionData)
s = 100
width = 300
pltSeg = directionData.iloc[s: s+width, :]
from matplotlib import pyplot as plt
plt.plot(pltSeg.index, pltSeg['direction_ope'], label="direction(op)")
plt.plot(pltSeg.index+1, pltSeg['direction_eye'],  label="direction(eye)")
plt.xticks(rotation=40)
plt.legend(loc="upper left", fontsize=14) 
plt.show()

# s = 100
# pltSeg = directionData.iloc[s: s+width, :]
# from matplotlib import pyplot as plt
# plt.plot(pltSeg.index, pltSeg['direction_ope'], label="direction_ope")
# plt.plot(pltSeg.index+1, pltSeg['direction_eye'],  label="direction_eye")
# plt.xticks(rotation=40)
# plt.legend(loc="upper left", fontsize=14) 
# plt.show()

# s = 1000
# pltSeg = directionData.iloc[s: s+width, :]
# from matplotlib import pyplot as plt
# plt.plot(pltSeg.index, pltSeg['direction_ope'], label="direction_ope")
# plt.plot(pltSeg.index+1, pltSeg['direction_eye'],  label="direction_eye")
# plt.xticks(rotation=40)
# plt.legend(loc="upper left", fontsize=14) 
# plt.show()

In [ ]:
# 平滑化の確認
a = directionData.iloc[100:300]
b = data1.iloc[100:300]
from matplotlib import pyplot as plt
plt.plot(a.index, a['centerEyeLookAtPoint-x'], label="after")
plt.plot(b.index, b['centerEyeLookAtPoint-x'], label="befor")
plt.xticks(rotation=40)
plt.legend(loc="upper left", fontsize=14) 
plt.show()

from matplotlib import pyplot as plt
plt.plot(a.index, a['centerEyeLookAtPoint-y'], label="after")
plt.plot(b.index, b['centerEyeLookAtPoint-y'], label="befor")
plt.xticks(rotation=40)
plt.legend(loc="upper left", fontsize=14) 
plt.show()

In [ ]:
# pypeのsegmentxyを使う場合（テストデータとトレインデータの分割でラベルの数を合わせられない）
# one_count = 0
# shift = 150
# for index, row in data.iterrows():
#     if row['intention'] == 1 and index-shift > 0:
#         one_count += 1  

In [ ]:
# pypeのsegmentxyを使う場合（テストデータとトレインデータの分割でラベルの数を合わせられない）
# count = 0
# data_ = pd.DataFrame()
# for index, row in data.iterrows():
#     if index-shift+1 >= 0:
#         if row['intention'] == 0 and count < one_count:
#             count += 1
#             data_ = data_.append(data.iloc[index-shift+1: index+1, :])
#         elif row['intention'] == 1:
#             data_ = data_.append(data.iloc[index-shift+1: index+1, :])
# data_ = data_.reset_index(drop=True)

In [ ]:
# pypeのsegmentxyを使う場合（ラベルの数を揃えて分割できなくてダメ）
# x = data_[['centerEyeLookAtPoint-x', 'centerEyeLookAtPoint-y']].values
# y = data_['nextClass'].values
# y = data_['intention'].values
# X = [x]
# Y = [y]
# X, Y, _ = seg_xy.transform(xs, ys)
# X_train, X_test, y_train, y_test = temporal_split(X, Y, test_size=0.3)

In [ ]:
#いろんな分類器を比較
# names = ["Logistic Regression", "Nearest Neighbors", 
#          "Linear SVM", "Polynomial SVM", "RBF SVM", "Sigmoid SVM", 
#          "Decision Tree","Random Forest", "AdaBoost", "Naive Bayes", 
#          "Linear Discriminant Analysis","Quadratic Discriminant Analysis"]

# classifiers = [
#     LogisticRegression(),
#     KNeighborsClassifier(),
#     SVC(kernel="linear"),
#     SVC(kernel="poly"),
#     SVC(kernel="rbf"),
#     SVC(kernel="sigmoid"),
#     DecisionTreeClassifier(),
#     RandomForestClassifier(),
#     AdaBoostClassifier(),
#     GaussianNB(),
#     LDA(),
#     QDA()]
# result = []
# for name, clf in zip(names, classifiers): # 指定した複数の分類機を順番に呼び出す
#     clf = Pype([
#                 ('features', FeatureRep(features=all_features())),  # extracts features
#                 ('rf', clf)  #  ML algorithm of sklearn
#             ])
#     clf.fit(X_train, y_train)
#     score1 = clf.score(X_train, y_train) # 正解率（train）の算出
#     score2 = clf.score(X_test, y_test) # 正解率（test）の算出
#     result.append([score1, score2]) # 結果の格納

# # test の正解率の大きい順に並べる
# df_result = pd.DataFrame(result, columns=['train', 'test'], index=names).sort_values('test', ascending=False)
# df_result # 結果の確認

In [ ]:
# df_result.plot(kind='bar', alpha=0.5, grid=True)


In [ ]:
# pred = clf.predict(X_test)
# cm = confusion_matrix(y_test, pred)
# sns.heatmap(cm)
# plt.tight_layout()
# plt.rcParams['figure.figsize'] = (20.0, 20.0)
# plt.show()

# create a pipeline
# clf = Pype([
#                 ('segment', SegmentXY(width=150, overlap=0, y_func=last)),  # segmentation
#                 ('features', FeatureRep()),  # extracts features
#                 ('rf', RandomForestClassifier(n_estimators=20))  #  ML algorithm of sklearn
#             ])
 
# print(clf)
# print(segmenty1)
# print(len([j for j, x in enumerate(segmenty2) if x == 28]))
# print(len([i for i, x in enumerate(segmenty2) if x == 1]))

In [ ]:
# 操作直前かそうでないかの分類
fts = {'var': variation, 'std': std, 'med': median}
classifier = RandomForestClassifier(n_estimators=20)
clfI1 = calssification(fts, classifier, segIx1, segIy1)
# モデルを保存する
filename = './model/home_clfI1.sav'
pickle.dump(clfI1, open(filename, 'wb'))

In [ ]:
# 操作座標の分類
fts = {'mean': mean, 'med': median, 'var': variation}
classifier = RandomForestClassifier(n_estimators=20)
clf1 = calssification(fts, classifier, segx1, segy1)
# モデルを保存する
filename = './model/home_clf1.sav'
pickle.dump(clf1, open(filename, 'wb'))

In [ ]:
# print(len(segy))
scoreI = clfI1.score(segIx1_, segIy1_)
print("count of data：{0}".format(len(segIx1_)))
print("clfI1_score: {0}".format(scoreI))
heatmap(clfI1, segIx1_, segIy1_)
score = clfI1.score(segx1_, segy1_)
print("count of data：{0}".format(len(segx1_)))
print("clf1_score: {0}".format(score))
heatmap(clf1, segx1_, segy1_)

In [ ]:
# 特徴量ごとのスコア（操作直前かそうでないかの分類）
fts_name = ['min', 'max', 'mean', 'std', 'kurt', 'meand', 'med', 'var']
fts_name_ = ['最小値', '最大値', '平均', '標準偏差', '尖度',  '平均時間微分', '中央値', '変動係数']
fts_ = [minimum, maximum, mean, std, kurt, mean_diff, median, variation]
ftsScore(fts_name, fts_name_, fts_, classifier, segIx4_0, segIy4_0)

In [ ]:
# 特徴量ごとのスコア（操作座標の分類）
fts_name = ['min', 'max', 'mean', 'std', 'kurt', 'meand', 'med', 'var']
fts_name_ = ['最小値', '最大値', '平均', '標準偏差', '尖度',  '平均時間微分', '中央値', '変動係数']
fts_ = [minimum, maximum, mean, std, kurt, mean_diff, median, variation]
ftsScore(fts_name, fts_name_, fts_, classifier, segx4_0, segy4_0)

In [ ]:
fts = {'min': minimum, 'max': maximum, 'med': median}
classifier = RandomForestClassifier(n_estimators=20)
clfI2 = calssification(fts, classifier, segIx2, segIy2)
filename = './model/home_clfI2.sav'
pickle.dump(clfI2, open(filename, 'wb'))

In [ ]:
# fts_name = ['min', 'max', 'mean', 'std', 'kurt', 'meand', 'med', 'var']
# fts_name_ = ['最小値', '最大値', '平均', '標準偏差', '尖度',  '平均時間微分', '中央値', '変動係数']
# fts_ = [minimum, maximum, mean, std, kurt, mean_diff, median, variation]
# ftsScore(fts_name, fts_name_, fts_, classifier, segx2, segy2)

In [ ]:
fts = {'min': minimum, 'max': maximum, 'med': median}
classifier = RandomForestClassifier(n_estimators=20)
clf2 = calssification(fts, classifier, segx2, segy2)
filename = './model/home_clf2.sav'
pickle.dump(clf2, open(filename, 'wb'))

In [ ]:
scoreI = clfI2.score(segIx2_, segIy2_)
print("count of data：{0}".format(len(segIx2_)))
print("clfI2_score: {0}".format(scoreI))
heatmap(clfI2, segIx2_, segIy2_)
print("count of data：{0}".format(len(segx2_)))
print("clf2_score: {0}".format(score))
scoreI = clf2.score(segx2_, segy2_)
heatmap(clf2, segx2_, segy2_)


# max_score = 0
# SearchMethod = 0
# RFC_grid = {RandomForestClassifier(): {"n_estimators": [i for i in range(1, 21)],
#                                        "criterion": ["gini", "entropy"],
#                                        "max_depth":[i for i in range(1, 5)],
#                                        "random_state": [i for i in range(0, 101)]
#                                       }}

# #ランダムフォレストの実行
# for model, param in tqdm(RFC_grid.items()):
#     clf = Pype([
#                 ('features', FeatureRep(features=all_features())),  # extracts features
#                 ('rf', GridSearchCV(model, param))  #  ML algorithm of sklearn
#             ])
#     clf.fit(X_train, y_train)
#     pred_y = clf.predict(X_test)
#     score = f1_score(y_test, pred_y, average="micro")

#     if max_score < score:
#         max_score = score
#         best_param = clf.get_params
#         best_model = model

# print("ベストスコア:{}".format(max_score))
# print("モデル:{}".format(best_model))
# print("パラメーター:{}".format(best_param))

In [ ]:
# 時間かかる
# clf = Pype([
#                 ('segment', SegmentXY(width=200, overlap=0.8, y_func=last)),  # segmentation
#                 ('features', FeatureRep()),  # extracts features
#                 ('rf', classifiers[2])  #  ML algorithm of sklearn
#             ])

# # fit and score
# clf.fit(X_train, y_train)

# score = clf.score(X_test, y_test)
# print("score of test data：", score)
# score = clf.score(X_train, y_train)
# print("score of train data：", score)

# score = clf.score(segmentx, segmenty)
# print("score of all data：", score)

In [ ]:
# data.iloc[60,:]['timestamp'] - data.iloc[0,:]['timestamp']
# print(data.iloc[0:10,:])
# 約1秒ずらし
demoIx = []
demoIy = []
demox = []
demoy = []
pre_processing(0, csv_datas, 'Class_demo', demoIx, demoIy, demox,  demoy, 2, 2, 60, 150)
fts = {'meand': mean_diff, 'mean': mean, 'var': variation}
classifier = RandomForestClassifier(n_estimators=20)
calssification(fts, classifier, demoIx, demoIy)
calssification(fts, classifier, demox, demoy)

In [ ]:
# data.iloc[300,:]['timestamp'] - data.iloc[0,:]['timestamp']
# 約5秒ずらし
demoIx2 = []
demoIy2 = []
demox2 = []
demoy2 = []
pre_processing(0, csv_datas, 'Class_demo2', demoIx2, demoIy2, demox2,  demoy2, 2, 2, 300, 150)
fts = {'meand': mean_diff, 'mean': mean, 'var': variation}
classifier = RandomForestClassifier(n_estimators=20)
calssification(fts, classifier, demoIx2, demoIy2)
calssification(fts, classifier, demox2, demoy2)

In [ ]:
# demo.iloc[480:500, :]

In [ ]:
# data.iloc[500,:]['timestamp'] - data.iloc[0,:]['timestamp']
# 約10秒ずらし
demoIx3 = []
demoIy3 = []
demox3 = []
demoy3 = []
pre_processing(0, csv_datas, 'Class_demo3', demoIx3, demoIy3, demox3,  demoy3, 2, 2, 500, 150)
fts = {'meand': mean_diff, 'mean': mean, 'var': variation}
classifier = RandomForestClassifier(n_estimators=20)
clf = calssification(fts, classifier, demoIx3, demoIy3)
clf = calssification(fts, classifier, demox3, demoy3)

In [ ]:
# demoseg_ = demo.iloc[340:490].copy()
# demoseg = demoseg_[['centerEyeLookAtPoint-x', 'centerEyeLookAtPoint-y']].values
# # demo_p = clf.predict(demoseg)
# demoseg
# print(clf)